### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()
tf.set_random_seed = 40


### Collect Data

In [0]:
import pandas as pd

In [26]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = pd.read_csv('/content/drive/My Drive/deep learning/R6_Internal_lab/prices.csv')

### Check all columns in the dataset

In [28]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [29]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
columns = ['date','symbol']
data.drop(columns,axis=1,inplace=True)

In [32]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
data = data.head(1000)

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X= data.drop("close",axis=1)
y= data.close
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 5)

In [35]:
y_test = pd.DataFrame(y_test)
y_test.shape

(300, 1)

In [36]:
y_train = pd.DataFrame(y_train)
y_train.shape

(700, 1)

## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')



In [0]:
#Normalize the data
x_n = tf.layers.batch_normalization(x,training=True)

#Actual Prices
y_ = tf.placeholder(shape=[None,1],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

3.Prediction

In [0]:
#We will use normalized data
#y = tf.add(tf.matmul(x,W),b,name='output')
y = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.003).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [44]:
y_train.shape

(700, 1)

In [45]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:X_train, 
                                                        y_:y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

('Training loss at step: ', 0, ' is ', 7946.0557)
('Training loss at step: ', 10, ' is ', 1.8029771)
('Training loss at step: ', 20, ' is ', 0.7103872)
('Training loss at step: ', 30, ' is ', 0.673917)
('Training loss at step: ', 40, ' is ', 0.64368767)
('Training loss at step: ', 50, ' is ', 0.6183752)
('Training loss at step: ', 60, ' is ', 0.5969869)
('Training loss at step: ', 70, ' is ', 0.5787726)
('Training loss at step: ', 80, ' is ', 0.56314975)
('Training loss at step: ', 90, ' is ', 0.54966605)


In [46]:
X_train.shape

(700, 4)

In [47]:

for epoch in range(100):
            
    #Calculate train_op and loss
    _, test_loss = sess.run([train_op,loss],feed_dict={x:X_test, 
                                                        y_:y_test})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', test_loss)

('Training loss at step: ', 0, ' is ', 5.6145782)
('Training loss at step: ', 10, ' is ', 0.42224005)
('Training loss at step: ', 20, ' is ', 0.41516948)
('Training loss at step: ', 30, ' is ', 0.40894142)
('Training loss at step: ', 40, ' is ', 0.40343666)
('Training loss at step: ', 50, ' is ', 0.3985586)
('Training loss at step: ', 60, ' is ', 0.39422303)
('Training loss at step: ', 70, ' is ', 0.3903603)
('Training loss at step: ', 80, ' is ', 0.3869119)
('Training loss at step: ', 90, ' is ', 0.3838277)


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [48]:
print(sess.run(W))
print(sess.run(W).shape)

[[ 5.288594  ]
 [ 5.3056335 ]
 [ 5.3091273 ]
 [-0.18218978]]
(4, 1)


In [49]:
print(sess.run(b))
print(sess.run(b).shape)

[2.6861594]
(1,)


### Find the Absolute mean square loss difference between training and testing loss.

In [50]:
import numpy as np

np.absolute(train_loss-test_loss)

0.15773928

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

model.add(tf.keras.layers.Dense(1))
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.05,clipnorm=1)

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer=sgd_optimizer, loss='mse')

### Execute the model

In [52]:
model.fit(X_train, y_train, epochs=100)


Epoch 1/100
700/700 [==============================] - 0s 333us/step - loss: 7402.4052
Epoch 2/100
700/700 [==============================] - 0s 106us/step - loss: 6293.6444
Epoch 3/100
700/700 [==============================] - 0s 109us/step - loss: 4803.0097
Epoch 4/100
700/700 [==============================] - 0s 109us/step - loss: 3381.1954
Epoch 5/100
700/700 [==============================] - 0s 108us/step - loss: 1812.1398
Epoch 6/100
700/700 [==============================] - 0s 104us/step - loss: 954.8525
Epoch 7/100
700/700 [==============================] - 0s 109us/step - loss: 641.6112
Epoch 8/100
700/700 [==============================] - 0s 109us/step - loss: 626.1160
Epoch 9/100
700/700 [==============================] - 0s 111us/step - loss: 666.7531
Epoch 10/100
700/700 [==============================] - 0s 125us/step - loss: 693.2882
Epoch 11/100
700/700 [==============================] - 0s 106us/step - loss: 831.3866
Epoch 12/100
700/700 [=========================

In [0]:
#y_pred = model.predict(X_test)

In [58]:
#type(y_test)

pandas.core.frame.DataFrame

In [62]:
#type(y_pred)

numpy.ndarray

In [0]:
#y_pred = pd.DataFrame(y_pred)

#y_test = y_test.reshape(-1,)
#mse = np.mean(np.square(y_pred-y_test))

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
import pandas as pd
iris_df = pd.read_csv('/content/drive/My Drive/deep learning/R6_Internal_lab/Iris.csv')

### Splitting the data into feature set and target set

In [84]:
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [85]:
X_i = iris_df.drop("Species", axis=1)
y_i = iris_df.Species
X_i.shape

(150, 5)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [86]:
from sklearn.preprocessing import LabelEncoder

iris_df = pd.get_dummies(iris_df,prefix ='Species')
iris_df.head()


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Divide the dataset into Training and test (70:30)

In [0]:
col = []

y_i = iris_df[['Species_Iris-setosa','Species_Iris-versicolor', 'Species_Iris-virginica']]


In [92]:
y_i = pd.DataFrame(y_i)
X_i.shape


(150, 5)

In [0]:
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X_i,y_i,test_size=0.3,random_state =15)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape((784,),input_shape=(28,28,)))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

In [0]:
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, input_shape=(4,), activation='sigmoid'))

#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))


In [0]:
#Add OUTPUT layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

### Fitting the model and predicting 

In [0]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [98]:
#X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X_i,y_i,test_size=0.3,random_state =15)
model.fit(X_train_i.values,y_train_i.values,          
          validation_data=(X_test_i.values,y_test_i.values),
          epochs=30,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/30
105/105 [==============================] - 0s 4ms/step - loss: 1.1630 - acc: 0.3333 - val_loss: 1.1590 - val_acc: 0.3333
Epoch 2/30
105/105 [==============================] - 0s 245us/step - loss: 1.1447 - acc: 0.3333 - val_loss: 1.1483 - val_acc: 0.3333
Epoch 3/30
105/105 [==============================] - 0s 235us/step - loss: 1.1354 - acc: 0.3333 - val_loss: 1.1365 - val_acc: 0.3333
Epoch 4/30
105/105 [==============================] - 0s 236us/step - loss: 1.1253 - acc: 0.3333 - val_loss: 1.1268 - val_acc: 0.3333
Epoch 5/30
105/105 [==============================] - 0s 246us/step - loss: 1.1176 - acc: 0.3333 - val_loss: 1.1181 - val_acc: 0.3333
Epoch 6/30
105/105 [==============================] - 0s 264us/step - loss: 1.1120 - acc: 0.3333 - val_loss: 1.1137 - val_acc: 0.3333
Epoch 7/30
105/105 [==============================] - 0s 229us/step - loss: 1.1081 - acc: 0.3333 - val_loss: 1.1069 - val_acc: 0.3333
Epoch 8/30
105/105 

### Report Accuracy of the predicted values